# R02. Stack Shares
- This calculates the shares of each stack in player results
- Type: Research
- Run Frequency: Irregular
- Sources:
    - A01. DraftKings
- Created: 3/2/2025
- Updated: 3/2/2025

### Imports

In [1]:
%run "U1. Imports.ipynb"
%run "U2. Utilities.ipynb"
%run "U3. Classes.ipynb"
%run "U4. Datasets.ipynb"
%run "U5. Models.ipynb"

### Settings

In [2]:
# Some random games to consider
contestKey_list = ['167658792', '167014090', '165405018', '165405324', '164017250', '163044558', '162458798']
draftGroupId_list = ['114004', '113232', '112056', '111836', '110628', '109187', '108160']

##### Merge player teams onto standings

In [3]:
standings_df_list = []
for i in range(len(contestKey_list)):
    draftGroupId = draftGroupId_list[i]
    contestKey = contestKey_list[i]
    # Read in draftables
    draftable_df = pd.read_csv(os.path.join(baseball_path, "A01. DraftKings", "2. Draftables", f"Draftables {draftGroupId}.csv"), encoding='iso-8859-1', usecols=['Name', 'TeamAbbrev'])
    # Read in standings
    position_list = ['P', 'P.1', 'C', '1B', '2B', '3B', 'SS', 'OF', 'OF.1', 'OF.2']
    standings_df = pd.read_csv(os.path.join(baseball_path, "A01. DraftKings", "5. Entry Results", f"Entry Results {contestKey}.csv"), encoding='iso-8859-1', usecols=position_list)
    # Merge player teams onto standings
    for position in position_list:
        standings_df = standings_df.merge(draftable_df, left_on=[position], right_on=['Name'], how='left', suffixes=("", f"_{position}"))

    standings_df_list.append(standings_df)

all_standings_df = pd.concat(standings_df_list)

##### Calculate Stack Shares

In [4]:
# Assuming df is your DataFrame
def row_value_counts(row):
    counts = row.value_counts().sort_values(ascending=False)
    return '-'.join(map(str, counts.values))

all_standings_df['counts'] = all_standings_df[['TeamAbbrev_C', 'TeamAbbrev_1B', 'TeamAbbrev_2B', 'TeamAbbrev_3B', 'TeamAbbrev_SS', 
                                               'TeamAbbrev_OF', 'TeamAbbrev_OF.1', 'TeamAbbrev_OF.2']].apply(row_value_counts, axis=1)

##### All

In [5]:
all_standings_df['counts'].value_counts(normalize=True)

counts
5-2-1              0.232491
5-3                0.149243
5-1-1-1            0.113260
4-3-1              0.069752
4-2-1-1            0.069220
2-1-1-1-1-1-1      0.058329
2-2-1-1-1-1        0.051217
3-2-1-1-1          0.048492
4-1-1-1-1          0.034528
4-4                0.030989
3-1-1-1-1-1        0.026722
1-1-1-1-1-1-1-1    0.025006
3-2-2-1            0.022910
3-3-1-1            0.021716
2-2-2-1-1          0.019219
4-2-2              0.012454
3-3-2              0.011444
2-2-2-2            0.002324
6-1-1              0.000402
6-2                0.000282
Name: proportion, dtype: float64

##### Top 5 out of 100%

In [39]:
all_standings_df[all_standings_df['counts'].isin(['5-2-1', '5-3', '5-1-1-1', '4-3-1', '4-2-1-1'])]['counts'].value_counts(normalize=True)

counts
5-2-1      0.366725
5-3        0.235412
5-1-1-1    0.178653
4-3-1      0.110024
4-2-1-1    0.109185
Name: proportion, dtype: float64